In [1]:
!pip3 install -U sec-edgar-downloader
!pip3 install -q -U google-generativeai
!pip3 install -q -U google-generativeai --upgrade
!pip3 install python-dotenv
!pip3 install fpdf
!pip3 install pip --upgrade
!pip3 install pyopenssl --upgrade

In [2]:
from sec_edgar_downloader import Downloader

# Download filings to the current working directory
dl = Downloader("Georgia Tech", "dsharma97@gatech.edu", ".")

In [3]:
# Get the latest supported filings, if available, for a
# specified list of tickers and CIKs
equity_ids = ["AAPL", "MSFT", "GOOG"]
for equity_id in equity_ids:
    dl.get("10-K", equity_id, after='1994-12-31', before='2024-01-01')

In [4]:
# import os
# import textwrap
# from fpdf import FPDF

# def text_to_pdf(text, filename):
#     a4_width_mm = 210
#     pt_to_mm = 0.35
#     fontsize_pt = 10
#     fontsize_mm = fontsize_pt * pt_to_mm
#     margin_bottom_mm = 10
#     character_width_mm = 7 * pt_to_mm
#     width_text = a4_width_mm / character_width_mm

#     pdf = FPDF(orientation='P', unit='mm', format='A4')
#     pdf.set_auto_page_break(True, margin=margin_bottom_mm)
#     pdf.add_page()
#     pdf.set_font(family='Courier', size=fontsize_pt)
#     splitted = text.split('\n')

#     for line in splitted:
#         lines = textwrap.wrap(line, width_text)

#         if len(lines) == 0:
#             pdf.ln()

#         for wrap in lines:
#             pdf.cell(0, fontsize_mm, wrap, ln=1)

#     pdf.output(filename, 'F')


# for company_ticker in os.listdir("./sec-edgar-filings"):
#     directory = "./sec-edgar-filings/{}/10-K".format(company_ticker)
#     for folder in os.listdir(directory):
#         folder_path = os.path.join(directory, folder)
#         filename = os.listdir(folder_path)[0]
#         file_path = os.path.join(folder_path, filename)
#         if filename.endswith('.txt'):
#             print(filename)
#             pdf_file = file_path[:-4]+".pdf"
#             file = open(file_path)
#             text = file.read()
#             file.close()
#             text_to_pdf(text, pdf_file)

In [5]:
import pathlib
import textwrap
import google.generativeai as genai
import os

from dotenv import load_dotenv
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

load_dotenv()

genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")


In [6]:
%%time
response = model.generate_content("What is the meaning of life?")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "The meaning of life is a complex and multifaceted question that has been pondered by philosophers and theologians for centuries. There is no one definitive answer, as the meaning of life is subjective and can vary greatly from person to person. \n\nHere are a few perspectives on the meaning of life:\n\n* **Philosophical:** Many philosophers believe that the meaning of life is to find happiness and fulfillment. This can be achieved through various means, such as pursuing knowledge, building relationships, creating art, or helping others.\n* **Religious:** Many religions believe that the meaning of life is to serve God or a higher power. This may involve following religious teachings, participating in religious rituals, or living a moral life.\n* **Existentialist:** Existentialists believe that life has no inherent meaning and that e

In [7]:
to_markdown(response.text)

> The meaning of life is a complex and multifaceted question that has been pondered by philosophers and theologians for centuries. There is no one definitive answer, as the meaning of life is subjective and can vary greatly from person to person. 
> 
> Here are a few perspectives on the meaning of life:
> 
> * **Philosophical:** Many philosophers believe that the meaning of life is to find happiness and fulfillment. This can be achieved through various means, such as pursuing knowledge, building relationships, creating art, or helping others.
> * **Religious:** Many religions believe that the meaning of life is to serve God or a higher power. This may involve following religious teachings, participating in religious rituals, or living a moral life.
> * **Existentialist:** Existentialists believe that life has no inherent meaning and that each individual must create their own meaning. This can be done through making choices, taking responsibility for one's actions, and living authentically.
> * **Humanistic:** Humanists believe that the meaning of life is to live a good life and make the world a better place. This may involve pursuing personal growth, helping others, and working towards social justice.
> 
> Ultimately, the meaning of life is up to each individual to decide. It is a journey of self-discovery and exploration. 
> 
> Here are some tips for finding meaning in your life: 
> 
> * **Reflect on your values and beliefs.** What is important to you? What do you believe in? 
> * **Explore your interests and passions.** What are you curious about? What do you enjoy doing? 
> * **Connect with others.** Build meaningful relationships with family, friends, and your community.
> * **Contribute to something larger than yourself.** Volunteer your time, donate to a worthy cause, or work towards making a difference in the world.
> * **Live in the present moment.** Appreciate the little things and find joy in everyday experiences.
> 
> Remember, the meaning of life is not a destination, but a journey. It is something that you create through your choices, actions, and experiences. 


In [24]:
import os
import re

def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

company_ticker = 'AAPL'

directory = "./sec-edgar-filings/{}/10-K".format(company_ticker)

parts = ["Using the following 29 years worth of SEC-Edgar 10-K fillings for the company {}, give me specific analytics and insights about the company: financial analysis (to track metrics like revenue over time), sentiment analysis, and specific observations.".format(company_ticker)]


for folder in os.listdir(directory):
    folder_path = directory+"/"+folder
    filename = os.listdir(folder_path)[0]
    print(filename)
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            year = file.readline()[-9:]
        text = "SEC Filling for Year {} - Inputted in HTML formatting:\n".format(year)
        print(folder)
        with open(file_path) as myfile:
            content = myfile.read()
        filetxt = read_text_from_file(file_path)

        analyze = re.search(r'Company Stock Performance.*?Operating Expenses', content, re.DOTALL)
        if not analyze:
            analyze = re.search(r'Company Stock Performance.{20000}', content, re.DOTALL)
        if not analyze:
            analyze = re.search(r"Item *5.{20000}", content, re.DOTALL)

        analyze = analyze.group()[:20000]
        print(analyze)
        text += analyze
        #text += read_text_from_file(file_path)
        #upload_filename = "{}-{}:".format(company_ticker, year)
        #print(upload_filename)
        #f = genai.upload_file(file_path, display_name=upload_filename)
        #print(f)
        parts.append(text)

full-submission.txt
0000320193-17-000070
Company Stock Performance</font></div><div style="line-height:120%;padding-top:8px;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;">The following graph shows a comparison of cumulative total shareholder return, calculated on a dividend reinvested basis, for the Company, the S&amp;P 500 Index, the S&amp;P Information Technology Index and the Dow Jones U.S. Technology Supersector Index for the five years ended </font><font style="font-family:Helvetica,sans-serif;font-size:9pt;">September&#160;30, 2017</font><font style="font-family:Helvetica,sans-serif;font-size:9pt;">. The graph assumes $100 was invested in each of the Company&#8217;s common stock, the S&amp;P 500 Index, the S&amp;P Information Technology Index and the Dow Jones U.S. Technology Supersector Index as of the market close on September&#160;28, 2012. Note that historic stock price performance is not necessarily indicative of future stock

In [25]:
messages = [
    {'role':'user',
     'parts': parts}
]
print(parts)
response = model.generate_content(messages, request_options={"timeout": 600})
to_markdown(response.text)

['Using the following 29 years with of SEC-Edgar 10-K fillings for the company AAPL, give me specific analytics and insights about the company: financial analysis (to track metrics like revenue over time), sentiment analysis, and specific observations. Give a simple visualization of the data as well.', 'SEC Filling for Year 20171103\n - Inputted in HTML formatting:\nCompany Stock Performance</font></div><div style="line-height:120%;padding-top:8px;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;">The following graph shows a comparison of cumulative total shareholder return, calculated on a dividend reinvested basis, for the Company, the S&amp;P 500 Index, the S&amp;P Information Technology Index and the Dow Jones U.S. Technology Supersector Index for the five years ended </font><font style="font-family:Helvetica,sans-serif;font-size:9pt;">September&#160;30, 2017</font><font style="font-family:Helvetica,sans-serif;font-size:9pt;">. The grap

> ## Analyzing 29 Years of Apple's 10-K Filings: Insights and Observations
> 
> Analyzing the provided snippets of Apple's 10-K filings from 1995 to 2023 offers valuable insights into the company's evolution, financial performance, and market position. While a comprehensive analysis would require access to the complete filings, some key observations can be made:
> 
> **Financial Analysis:**
> 
> * **Revenue Growth:** Apple has experienced remarkable revenue growth over the 29 years. Starting with $11.06 billion in 1995, revenue reached a peak of $383.3 billion in 2023, demonstrating significant expansion.
> * **Profitability:** Net income has also shown an upward trend, although with some fluctuations due to factors like restructuring costs and investments. The company has maintained consistent profitability since the late 1990s, with 2023 seeing a net income of $97.0 billion.
> * **Product Mix:** The company has strategically shifted its product mix over time. Early filings focus on Macintosh computers, while later filings highlight the iPhone, iPad, and services like the App Store and Apple Music as significant revenue drivers. 
> * **International Expansion:** International sales have played a growing role in Apple's success. From representing 48% of net sales in 1995, international markets have become increasingly important, although specific regional breakdowns would require further data.
> 
> **Sentiment Analysis:**
> 
> * **Early Challenges:** The filings from the late 1990s reflect a period of uncertainty and restructuring for Apple.  There were concerns about market share, competition, and financial performance.
> * **Return to Growth and Innovation:** The tone shifts significantly in the 2000s with the introduction of the iPod and iPhone. The language becomes more confident, emphasizing innovation, market leadership, and strong financial results. 
> * **Focus on Services:** Recent filings highlight the importance of services in Apple's overall strategy. The company emphasizes its ecosystem of devices, software, and services and its growing recurring revenue streams.
> 
> **Specific Observations:**
> 
> * **Competition:** Apple consistently acknowledges the competitive landscape of its markets, mentioning companies like Microsoft and mobile communication device manufacturers.
> * **Research and Development:** The company emphasizes the importance of R&D for its future growth and competitive position. Filings indicate ongoing investment in developing new products and technologies.
> * **Risk Factors:** Apple identifies various risk factors, including dependence on new product introductions, component shortages, and economic conditions.
> 
> **Visualization:**
> 
> While the provided snippets include snapshots of Apple's stock performance compared to various indices, a more comprehensive visualization could include:
> 
> * **Revenue and Net Income:** A line chart demonstrating the growth of Apple's revenue and net income over the 29 years. 
> * **Product Segment Revenue:** A stacked bar chart showing the revenue breakdown by product segment (iPhone, Mac, iPad, Services, etc.) over time to visualize the changing product mix.
> * **International Sales:** A map or chart showing the geographic distribution of Apple's revenue to highlight the importance of international markets.
> 
> **Limitations:**
> 
> The analysis above is based on limited excerpts from the 10-K filings. A complete understanding would require access to the full filings and potentially other data sources.  
> 
> **Conclusion:**
> 
> Apple's 10-K filings provide a fascinating look into the company's journey from a struggling computer manufacturer to a global tech giant. The documents reveal a consistent commitment to innovation, a strategic shift in product focus, and successful expansion into international markets and services. While challenges remain, Apple's financial performance and market position suggest continued success in the years to come.


> ## Apple Inc. (AAPL): Analysis of SEC Filings from 1995 to 2023
> 
> Based on the provided excerpts from AAPL's 10-K filings, here's an overview of the company's performance and sentiment over time:
> 
> **Financial Analysis:**
> 
> * **Revenue:** Apple experienced significant growth in the late 1990s and early 2000s, driven by the success of the iMac and later, the iPod. The introduction of the iPhone in 2007 marked another turning point, propelling the company to even greater heights. In recent years, revenue growth has stabilized but remains substantial, with Services becoming an increasingly important revenue stream.
> * **Net Income:** Similar to revenue, net income followed a similar trajectory, demonstrating profitability alongside growth. However, there were periods of losses in the late 1990s due to restructuring and challenges in the personal computer market.
> * **Earnings Per Share (EPS):** EPS trends closely mirror net income, reflecting the company's profitability on a per-share basis. 
> * **Cash and Investments:** Apple's cash and short-term investments have steadily increased over the years, indicating strong financial health and the ability to invest in future growth opportunities.
> * **Total Assets and Liabilities:** Both total assets and liabilities have grown substantially, reflecting the expansion of the company's operations and financial activities. 
> 
> **Sentiment Analysis:**
> 
> * **Early Years (1995-2000):** Filings from this period express concerns about the highly competitive personal computer market, pricing pressures, and challenges related to product transitions. There was a focus on managing costs and improving efficiency. 
> * **Growth Era (2001-2013):** Sentiment becomes more positive as Apple experiences tremendous success with the iPod, iPhone, and iPad. The company emphasizes its innovation, design, and integrated digital lifestyle solutions. 
> * **Mature Phase (2014-2023):** Filings acknowledge the mature nature of some of its core markets but remain optimistic about future growth prospects. The company highlights the importance of Services and its expanding ecosystem.
> 
> **Specific Observations:**
> 
> * **Shift in Product Mix:** The filings show a clear shift in Apple's product mix over time, from primarily focusing on Macintosh computers to a more diversified portfolio encompassing iPhones, iPads, wearables, and services.
> * **International Expansion:** Apple has consistently emphasized the importance of international markets, with international sales constituting a significant portion of its total revenue.
> * **Competitive Landscape:** The company acknowledges the intense competition it faces in various markets, particularly from other technology giants and emerging players. 
> * **Focus on Innovation:** Innovation remains a core theme throughout the filings, with Apple consistently highlighting its commitment to research and development and bringing new products and technologies to market.
> 
> **Limitations:**
> 
> * **This analysis is based on excerpts and summaries of the 10-K filings and may not capture the full complexity of the company's financial performance and sentiment.**
> * **The filings only provide information up to November 2023, so any subsequent developments are not reflected in this analysis.**
> 
> **Overall, Apple's SEC filings portray a company that has navigated a challenging and competitive landscape to become one of the most valuable and influential technology companies in the world. While challenges and uncertainties remain, Apple's strong financial position, focus on innovation, and loyal customer base suggest continued success in the years to come.**
